In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm

def search_air(start_area, arrive_area, start_day, end_day=None, member={'adult':1,'youth':0,'baby':0}):
    '''
    start_area : 출발지를 입력하세요
    arrive_area : 도착지를 입력하세요
    start_day : 출발하는 날짜를 입력하세요(YYYY.MM.DD)
    end_day : (선택) 돌아오는 날짜를 입력하세요(YYYY.MM.DD)
    함수를 진행 중 일부정보가 보여지고 종료시 데이터프레임이 리스트로 반환됩니다.
    '''
    res = []
    for s, a, d in tqdm([(start_area, arrive_area, start_day), (arrive_area, start_area, end_day)]):
        if d == None:
            break
        
        options=Options()
        options.add_experimental_option('detach',True) #화면이 꺼지지 않고 유지
        options.add_argument('--start-maximized') # 화면을 최대화
        service=Service(ChromeDriverManager().install()) #크롬드라이버 자동 설치
        
        driver=webdriver.Chrome(service=service, options=options)
        

        # ✨ 기본 검색창 설정
        url = 'https://flight.naver.com/'
        
        driver.get(url)
        time.sleep(1)

        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[1]/button[2]/i').click()
        time.sleep(1)

        # ✨ 출발지 설정
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[1]').click()
        time.sleep(1)
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[1]/div/input').send_keys(s)
        time.sleep(1)
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/section/div/a[1]').click()
        time.sleep(1)
        
        # ✨ 도착지 설정
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[2]').click()
        time.sleep(1)
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[1]/div/input').send_keys(a)
        time.sleep(1)
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/section/div/a[1]').click()
        time.sleep(1)
        
        # ✨ 달력 조작부
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[2]/button[1]').click()
        time.sleep(1)
        
        # ✨ 출발 날짜 선택
        s_calendar = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div')
        s_find_month = s_calendar.find_element(By.XPATH,f"//div[text()='{d[:8]}']")
        s_find_calendar = s_find_month.find_element(By.XPATH, ".//ancestor::div")
        s_find_calendar.find_element(By.XPATH,f"//b[text()='{d[8:]}']").click()
        time.sleep(1)
        
        # ✨ 인원 선택
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[3]/button').click()
        time.sleep(1)
        if member['adult'] > 1:
            for i in range(2, member['adult']+1):
                driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[4]/div/div/div[1]/div[1]/button[2]').click()
                time.sleep(1)
        if member['youth'] > 0:
            for i in range(1, member['youth']+1):
                driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[4]/div/div/div[1]/div[2]/button[2]').click()
                time.sleep(1)
        if member['baby'] > 0:
            for i in range(1, member['baby']+1):
                driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[4]/div/div/div[1]/div[3]/button[2]').click()
                time.sleep(1)
            
        # ✨ 검색시작
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/button[1]').click()
        time.sleep(1)
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/button[1]').click()
        time.sleep(1)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.sideBanner_section__Frd3t')))
        time.sleep(1)
        
        # ✨ 스크롤 내리기
        before_location = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(1)
            after_location = driver.execute_script("return document.body.scrollHeight")
            if before_location == after_location:
                break
            before_location = after_location

        air_list = driver.find_elements(By.CSS_SELECTOR,'div.domestic_Flight__8bR_b>div')
        
        go_res = []
        for i in air_list:
            try:
                tmp = {}
                item = i.text.split('\n')
                tmp['항공사'] = item[0]
                tmp['출발시간'] = item[1]
                tmp['도착시간'] = item[2]
                tmp['운항시간'] = item[3]
                tmp['클래스'] = item[4].split()[0]
                tmp['가격'] = item[4].split()[1]
                go_res.append(tmp)
            except:
                pass
        
        go_df = pd.DataFrame(go_res)
        print(go_df.shape)
        display(go_df.head())
        res.append(go_df)

        # ✨ 드라이버 종료
        driver.quit()
    return res

In [2]:
start_area = '서울'
arrive_area = '제주'
start_day = '2024.10.25'
end_day = '2024.10.27'
member = {
    'adult':3,
    'youth':2,
    'baby':0
}

res = search_air(start_area, arrive_area, start_day, end_day, member)
display(res[0])
display(res[1])

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

(96, 6)


,항공사,출발시간,도착시간,운항시간,클래스,가격
0,진에어,06:00GMP,07:10CJU,01시간 10분,할인석편도,"112,600원~"
1,에어서울,06:00GMP,07:10CJU,01시간 10분,할인석편도,"113,700원~"
2,진에어,06:00GMP,07:10CJU,01시간 10분,일반석편도,"119,700원~"
3,에어서울,06:00GMP,07:10CJU,01시간 10분,일반석편도,"120,800원~"
4,진에어,06:05GMP,07:15CJU,01시간 10분,할인석편도,"108,600원~"


 50%|██████████████████████████████████████████                                          | 1/2 [00:41<00:41, 41.33s/it]

(56, 6)


,항공사,출발시간,도착시간,운항시간,클래스,가격
0,아시아나항공,06:55CJU,08:10GMP,01시간 15분,특가석편도,"66,700원~"
1,아시아나항공,06:55CJU,08:10GMP,01시간 15분,할인석편도,"72,700원~"
2,아시아나항공,06:55CJU,08:10GMP,01시간 15분,일반석편도,"132,700원~"
3,아시아나항공,06:55CJU,08:10GMP,01시간 15분,비즈니스석편도,"189,700원~"
4,아시아나항공,07:10CJU,08:25GMP,01시간 15분,특가석편도,"66,700원~"


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:22<00:00, 41.02s/it]


,항공사,출발시간,도착시간,운항시간,클래스,가격
0,진에어,06:00GMP,07:10CJU,01시간 10분,할인석편도,"112,600원~"
1,에어서울,06:00GMP,07:10CJU,01시간 10분,할인석편도,"113,700원~"
2,진에어,06:00GMP,07:10CJU,01시간 10분,일반석편도,"119,700원~"
3,에어서울,06:00GMP,07:10CJU,01시간 10분,일반석편도,"120,800원~"
4,진에어,06:05GMP,07:15CJU,01시간 10분,할인석편도,"108,600원~"
...,...,...,...,...,...,...
91,이스타항공,19:25GMP,20:35CJU,01시간 10분,일반석편도,"119,600원~"
92,제주항공,19:30GMP,20:40CJU,01시간 10분,일반석편도,"119,100원~"
93,이스타항공,19:45GMP,20:55CJU,01시간 10분,일반석편도,"119,600원~"
94,아시아나항공,20:50GMP,22:00CJU,01시간 10분,일반석편도,"132,700원~"


,항공사,출발시간,도착시간,운항시간,클래스,가격
0,아시아나항공,06:55CJU,08:10GMP,01시간 15분,특가석편도,"66,700원~"
1,아시아나항공,06:55CJU,08:10GMP,01시간 15분,할인석편도,"72,700원~"
2,아시아나항공,06:55CJU,08:10GMP,01시간 15분,일반석편도,"132,700원~"
3,아시아나항공,06:55CJU,08:10GMP,01시간 15분,비즈니스석편도,"189,700원~"
4,아시아나항공,07:10CJU,08:25GMP,01시간 15분,특가석편도,"66,700원~"
5,아시아나항공,07:10CJU,08:25GMP,01시간 15분,할인석편도,"72,700원~"
6,아시아나항공,07:10CJU,08:25GMP,01시간 15분,일반석편도,"132,700원~"
7,아시아나항공,07:10CJU,08:25GMP,01시간 15분,비즈니스석편도,"189,700원~"
8,제주항공,07:40CJU,08:50GMP,01시간 10분,할인석편도,"103,500원~"
9,제주항공,07:40CJU,08:50GMP,01시간 10분,일반석편도,"119,100원~"
